# Measurement Grouping

Since current quantum hardware is limited to single-qubit projective measurement, only terms commuting within individual qubit's subspace can be measured together. These terms are said to be qubit-wise commuting (QWC). 

Thus, one can not measure the entire electronic Hamiltonian $\hat H$ at once, and instead needs to separate it into fragments. 
$$\hat H = \sum_n \hat H_n$$
where each $\hat H_n$ is a QWC fragment. 

In [1]:
from utility import * 

Here we use $LiH$ as an example for finding QWC fragments. Notice below that each fragment has the same terms on all qubits.  

To show differences between QWC and more advanced grouping, we didn't use the qubit-tappering techinique shown in step 2.

In [2]:
lih = get_qubit_hamiltonian(mol='lih', geometry=1, basis='sto-3g', qubit_transf='jw')

qwc_list = get_qwc_group(lih)
print('Fragments 1: \n{}\n'.format(qwc_list[4]))
print('Fragments 2:\n{}\n'.format(qwc_list[1]))
print('Number of fragments: {}'.format(len(qwc_list)))

Fragments 1: 
-0.0032403902911913025 [X0 Z1 Z2 Y3 Y4 Z5 Z6 Z7 Z8 Z9 Z10 X11] +
0.06558452315458399 [Z6 Z8] +
0.06980180803300672 [Z6 Z9] +
0.06980180803300672 [Z7 Z8] +
0.06558452315458399 [Z7 Z9]

Fragments 2:
-0.0009084689845792862 [Y0 Z1 Z2 Z3 Z4 Z5 Y6 Y7 Z8 Z9 Z10 Y11] +
0.09902225720340921 [Z1 Z8] +
0.09654955103502391 [Z1 Z9] +
0.06832924266505087 [Z2 Z8] +
0.07512476934747558 [Z2 Z9] +
0.07512476934747558 [Z3 Z8] +
0.06832924266505087 [Z3 Z9] +
0.060097029776676233 [Z4 Z8] +
0.07068762352443518 [Z4 Z9] +
0.07068762352443518 [Z5 Z8] +
0.060097029776676233 [Z5 Z9] +
-0.21675429323818415 [Z8] +
0.07823637778985215 [Z8 Z9] +
0.06428519917726606 [Z8 Z10] +
-0.21675429323818413 [Z9] +
0.06811818560946911 [Z9 Z10]

Number of fragments: 154


By applying extra unitaries, one may rotate more terms of $\hat H$ into a QWC fragment.  

Recall that in digital quantum computing, the expectation value of $\hat H_n$ given a trial wavefunction $|\psi\rangle$ is 
$$ E_n =\ \langle\psi| \hat H_n | \psi\rangle$$
Inserting unitary transformation $\hat U_n$ does not change the expectation value.
$$ E_n =\ \langle\psi| \hat U_n^\dagger \hat U_n \hat H_n \hat U_n^\dagger \hat U_n  |\psi\rangle$$ 
This nonetheless changes the trial wavefunction and the terms to be measured. 
$$ |\psi\rangle \rightarrow \hat U_n |\psi\rangle = |\phi\rangle$$
$$ \hat H_n \rightarrow \hat U_n \hat H_n \hat U_n^\dagger = \hat A_n$$
The transformation of $|\psi \rangle$ can be done on the quantum computer, and the transformation of $\hat H_n$ is possible on the classical computer. 

Now, although $\hat A_n$ needs to be a QWC fragment to be measurable on a quantum computer, $\hat H_n$ does not. 
Instead, if we restrict $\hat U_n$ to be a clifford operation, the terms in $\hat H$ need only mutually commute. 

Here, we obtain measurable parts of $LiH$ by partitioning its terms into mutually commuting fragments. 

In [12]:
comm_groups = get_commuting_group(lih)
print('Number of mutually commuting fragments: {}'.format(len(comm_groups)))
print('The first commuting group')
print(comm_groups[1])

Number of mutually commuting fragments: 36
The first commuting group
-3.93444195675791 [] +
0.007923321157850723 [X0 Y1 Y2 X3] +
-0.00015507836015035223 [X0 Z1 Z2 X3 Y4 Y5] +
-0.002077886902662328 [X0 Z1 Z2 X3 Y6 Y7] +
-0.0020778869026623274 [X0 Z1 Z2 X3 Y8 Y9] +
-0.0021028659073762156 [X0 Z1 Z2 X3 Y10 Y11] +
0.007923321157850723 [Y0 X1 X2 Y3] +
-0.00015507836015035223 [Y0 Z1 Z2 Y3 X4 X5] +
-0.002077886902662328 [Y0 Z1 Z2 Y3 X6 X7] +
-0.0020778869026623274 [Y0 Z1 Z2 Y3 X8 X9] +
-0.0021028659073762156 [Y0 Z1 Z2 Y3 X10 X11] +
0.11709373561826503 [Z0 Z3] +
-0.0001550783601503522 [X1 X2 X4 X5] +
-0.002077886902662328 [X1 X2 X6 X7] +
-0.0020778869026623274 [X1 X2 X8 X9] +
-0.0021028659073762156 [X1 X2 X10 X11] +
-0.0001550783601503522 [Y1 Y2 Y4 Y5] +
-0.002077886902662328 [Y1 Y2 Y6 Y7] +
-0.0020778869026623274 [Y1 Y2 Y8 Y9] +
-0.0021028659073762156 [Y1 Y2 Y10 Y11] +
0.11709373561826503 [Z1 Z2] +
-0.010590593747757844 [X4 X5 Y6 Y7] +
-0.010590593747757839 [X4 X5 Y8 Y9] +
-0.00673303078088139

To see this fragment is indeed measurable, one can construct the corresponding unitary operator $\hat U_n$.

## H4


In [3]:
h4 = get_qubit_hamiltonian(mol='h4', geometry=90, basis='sto-3g', qubit_transf='jw')

qwc_list = get_qwc_group(h4)
print('Fragments 1: \n{}\n'.format(qwc_list[4]))
print('Fragments 2:\n{}\n'.format(qwc_list[1]))
print('Number of fragments: {}'.format(len(qwc_list)))

comm_groups = get_commuting_group(h4)
print('Number of mutually commuting fragments: {}'.format(len(comm_groups)))
print('The first commuting group')
print(comm_groups[1])

Fragments 1: 
0.18909898661525393 [Z0] +
0.08362706285309376 [Z0 Z2] +
0.08362706285309385 [Z0 Z4] +
0.09753498283421291 [Z0 Z6] +
0.012952358031273271 [Y1 Z2 Y3 Y5 Z6 Y7] +
0.09422690650757803 [Z2 Z4] +
0.007958917262825431 [Z4] +
0.08288775222014044 [Z4 Z6]

Fragments 2:
-0.03426847228607793 [Y0 Z1 Z2 Y3 X4 Z5 Z6 X7] +
0.1174470883265534 [Z1 Z2] +
0.11843647604235702 [Z1 Z6] +
0.007958917262825008 [Z2] +
0.11597747867973224 [Z2 Z5] +
0.08288775222014037 [Z2 Z6] +
0.12047716405473605 [Z5 Z6] +
-0.1949783960025558 [Z6]

Number of fragments: 33
Number of mutually commuting fragments: 4
The first commuting group
-0.6427167034502669 [] +
-0.033820025473459636 [X0 X1 Y4 Y5] +
0.033820025473459636 [X0 Y1 Y4 X5] +
0.012952358031273264 [X0 Z1 Y2 Y4 Z5 X6] +
-0.03426847228607793 [X0 Z1 Z2 X3 Y4 Z5 Z6 Y7] +
0.03426847228607793 [X0 Z1 Z2 Y3 Y4 Z5 Z6 X7] +
0.033820025473459636 [Y0 X1 X4 Y5] +
-0.033820025473459636 [Y0 Y1 X4 X5] +
0.012952358031273264 [Y0 Z1 X2 X4 Z5 Y6] +
0.03426847228607793 [Y0 

## N2


In [4]:
n2 = get_qubit_hamiltonian(mol='n2', geometry=1.5, basis='sto-3g', qubit_transf='jw')

qwc_list = get_qwc_group(n2)
print('Fragments 1: \n{}\n'.format(qwc_list[4]))
print('Fragments 2:\n{}\n'.format(qwc_list[1]))
print('Number of fragments: {}'.format(len(qwc_list)))

comm_groups = get_commuting_group(n2)
print('Number of mutually commuting fragments: {}'.format(len(comm_groups)))
print('The first commuting group')
print(comm_groups[1])

Fragments 1: 
-0.02478426212035617 [X0 Z1 X2 Y3 Z4 Z5 Z6 Z7 Z8 Z9 Z10 Z11 Z12 Z13 Z14 Z15 Z16 Z17 Z18 Y19]

Fragments 2:
0.0018466431071313863 [Y0 Z1 Z2 Z3 Z4 Z5 Z6 Z7 Z8 Z9 Z10 Z11 Z12 Z13 Z14 Z15 Y16 Y17 Z18 Y19] +
0.5609229251606517 [Z1 Z2] +
0.08821851825203508 [Z1 Z3] +
9.4845255891763 [Z2] +
0.5612472046058594 [Z2 Z3] +
0.160957215282318 [Z2 Z4] +
0.16785191366744068 [Z2 Z5] +
0.15268765679475937 [Z2 Z6] +
0.16072252858842095 [Z2 Z7] +
0.15749618413582347 [Z2 Z8] +
0.16061613816948936 [Z2 Z9] +
0.15729967438386513 [Z2 Z10] +
0.1598494875605973 [Z2 Z11] +
0.15729967438386522 [Z2 Z12] +
0.15984948756059736 [Z2 Z13] +
0.16346016927197787 [Z2 Z14] +
0.16650523589696337 [Z2 Z15] +
0.185577681924766 [Z2 Z18] +
9.4845255891763 [Z3] +
0.16785191366744068 [Z3 Z4] +
0.160957215282318 [Z3 Z5] +
0.16072252858842095 [Z3 Z6] +
0.15268765679475937 [Z3 Z7] +
0.16061613816948936 [Z3 Z8] +
0.15749618413582347 [Z3 Z9] +
0.1598494875605973 [Z3 Z10] +
0.15729967438386513 [Z3 Z11] +
0.1598494875605973

## NH3

In [5]:
nh3 = get_qubit_hamiltonian(mol='nh3', geometry=1.5, basis='sto-3g', qubit_transf='jw')

qwc_list = get_qwc_group(nh3)
print('Fragments 1: \n{}\n'.format(qwc_list[4]))
print('Fragments 2:\n{}\n'.format(qwc_list[1]))
print('Number of fragments: {}'.format(len(qwc_list)))

comm_groups = get_commuting_group(nh3)
print('Number of mutually commuting fragments: {}'.format(len(comm_groups)))
print('The first commuting group')
print(comm_groups[1])



Fragments 1: 
-6.936302568531403e-06 [X0 Z1 Z2 Z3 Z4 Z5 Z6 Y7 Y8 Z9 Z10 Z11 Z12 Z13 Z14 X15]

Fragments 2:
-0.00010455100997998253 [X0 Z1 Z2 Z3 Z4 Z5 Z6 Z7 Z8 Z9 Z10 Z11 Z12 Y13 Y14 X15] +
0.14969891722954573 [Z1 Z7] +
0.20598064364339533 [Z1 Z8] +
0.12174210409401362 [Z2 Z7] +
0.11546045156076182 [Z2 Z8] +
0.09831098799691565 [Z3 Z7] +
0.14354630213537217 [Z3 Z8] +
0.09947179937625795 [Z4 Z7] +
0.1042766366416333 [Z4 Z8] +
0.08889341429331085 [Z5 Z7] +
0.113922944865534 [Z5 Z8] +
0.1206285695421506 [Z6 Z7] +
0.1042766366416331 [Z6 Z8] +
0.6438606242028247 [Z7] +
0.11392294486553409 [Z7 Z8] +
0.1042766366416331 [Z7 Z9] +
0.10634199576894605 [Z7 Z10] +
0.08960978422849084 [Z7 Z11] +
0.12290714305009653 [Z7 Z12] +
0.7457764202133409 [Z8] +
0.15281257952019495 [Z8 Z9] +
0.089986170842562 [Z8 Z10] +
0.10947239329894326 [Z8 Z11] +
0.11164511556016149 [Z8 Z12]

Number of fragments: 1200
Number of mutually commuting fragments: 152
The first commuting group
-35.264006651522 [] +
-0.01326918426